Q1. Write a Python program to scrape all available books from the website (https://books.toscrape.com/) Books to Scrape – a live site built for practicing scraping (safe, legal, no anti-bot). For each book, extract the following details:

Title
Price
Availability (In stock / Out of stock)
Star Rating (One, Two, Three, Four, Five)
Store the scraped results into a Pandas DataFrame and export them to a CSV file named books.csv.

(Note: Use the requests library to fetch the HTML page. Use BeautifulSoup to parse and extract book details and handle pagination so that books from all pages are scraped)

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# small helper dict for star ratings
star_map = {
    "One": "1",
    "Two": "2",
    "Three": "3",
    "Four": "4",
    "Five": "5"
}


title_list = []
price_list = []
stock_list = []
star_list = []


dataset_link = "https://books.toscrape.com/catalogue/page-{}.html"


for i in range(1, 51):
    url = dataset_link.format(i)
    res = requests.get(url)
    soup = BeautifulSoup(res.text, "html.parser")

    books = soup.find_all("article", class_="product_pod")

    for b in books:
        # title
        t = b.h3.a["title"]

        # price
        p = b.find("p", class_="price_color").text

        # availability
        s = b.find("p", class_="instock availability").text.strip()

        # star rating
        r_class = b.find("p", class_="star-rating")["class"][1]
        r = star_map.get(r_class, "0")

        # save to lists
        title_list.append(t)
        price_list.append(p)
        stock_list.append(s)
        star_list.append(r)

# make dataframe
df = pd.DataFrame({
    "Title": title_list,
    "Price": price_list,
    "Availability": stock_list,
    "Star_Rating": star_list
})

# save to csv
df.to_csv("books.csv", index=False)

print("Scraping done , data saved to books.csv")
df.head()


Scraping done , data saved to books.csv


,Title,Price,Availability,Star_Rating
0,A Light in the Attic,Â£51.77,In stock,3
1,Tipping the Velvet,Â£53.74,In stock,1
2,Soumission,Â£50.10,In stock,1
3,Sharp Objects,Â£47.82,In stock,4
4,Sapiens: A Brief History of Humankind,Â£54.23,In stock,5


Q2. Write a Python program to scrape the IMDB Top 250 Movies list (https://www.imdb.com/chart/top/) . For each movie, extract the following details:

Rank (1–250)
Movie Title
Year of Release
IMDB Rating
Store the results in a Pandas DataFrame and export it to a CSV file named imdb_top250.csv. (Note: Use Selenium/Playwright to scrape the required details from this website)


In [14]:

from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time, tempfile


cfg1 = webdriver.ChromeOptions()
cfg1.add_argument("--headless")
cfg1.add_argument("--no-sandbox")
cfg1.add_argument("--disable-dev-shm-usage")

tmpd = tempfile.mkdtemp()
cfg1.add_argument(f"--user-data-dir={tmpd}")

drv1 = webdriver.Chrome(options=cfg1)

url1 = "https://www.imdb.com/chart/top/"
drv1.get(url1)
time.sleep(3)               # wait for page to render
pg_html = drv1.page_source
drv1.quit()

sp1 = BeautifulSoup(pg_html, "html.parser")
out = []

trs = sp1.select("tbody.lister-list tr")
#print("table rows found:", len(trs))
if trs and len(trs) > 0:
    for r in trs:
        try:
            td = r.select_one("td.titleColumn")
            if not td:
                continue
            # rank
            rk = td.get_text(strip=True).split(".")[0]
            # title
            atag = td.select_one("a")
            nm = atag.text.strip() if atag else ""
            # year (strip parentheses)
            ytag = td.select_one("span.secondaryInfo")
            yr = ytag.text.strip("()") if ytag else ""
            # rating (some pages put it inside strong)
            rt_tag = r.select_one("td.ratingColumn.imdbRating strong") or r.select_one("strong")
            rt = rt_tag.text.strip() if rt_tag else ""
            out.append({"Rank": rk, "Title": nm, "Release": yr, "Rating": rt})
        except Exception:
            continue

if not out:
    lis = sp1.find_all("li", class_="ipc-metadata-list-summary-item")
    for it in lis:
        try:
            h3 = it.find("h3")
            if not h3:
                continue
            txt = h3.get_text(strip=True)
            #  Movie Name
            rk, nm = txt.split(". ", 1)
            # year
            ytag = it.find("span", class_="cli-title-metadata-item")
            yr = ytag.get_text(strip=True) if ytag else ""
            # rating
            rtag = it.find("span", class_="ipc-rating-star--rating")
            rt = rtag.get_text(strip=True) if rtag else ""
            out.append({"Rank": rk, "Title": nm, "Release": yr, "Rating": rt})
        except Exception:
            continue


if not out:
    print(" No movies scraped. HTML snippet ")
    print(pg_html[:1200])
else:

    df = pd.DataFrame(out)
    df.to_csv("imdb_top250.csv", index=False)

    for m in out:
        print(m)

    print("Total movies scraped ", len(out))


{'Rank': '1', 'Title': 'The Shawshank Redemption', 'Release': '1994', 'Rating': '9.3'}
{'Rank': '2', 'Title': 'The Godfather', 'Release': '1972', 'Rating': '9.2'}
{'Rank': '3', 'Title': 'The Dark Knight', 'Release': '2008', 'Rating': '9.1'}
{'Rank': '4', 'Title': 'The Godfather Part II', 'Release': '1974', 'Rating': '9.0'}
{'Rank': '5', 'Title': '12 Angry Men', 'Release': '1957', 'Rating': '9.0'}
{'Rank': '6', 'Title': 'The Lord of the Rings: The Return of the King', 'Release': '2003', 'Rating': '9.0'}
{'Rank': '7', 'Title': "Schindler's List", 'Release': '1993', 'Rating': '9.0'}
{'Rank': '8', 'Title': 'Pulp Fiction', 'Release': '1994', 'Rating': '8.8'}
{'Rank': '9', 'Title': 'The Lord of the Rings: The Fellowship of the Ring', 'Release': '2001', 'Rating': '8.9'}
{'Rank': '10', 'Title': 'The Good, the Bad and the Ugly', 'Release': '1966', 'Rating': '8.8'}
{'Rank': '11', 'Title': 'Forrest Gump', 'Release': '1994', 'Rating': '8.8'}
{'Rank': '12', 'Title': 'The Lord of the Rings: The Two 

Q3. Write a Python program to scrape the weather information for top world cities from the given website (https://www.timeanddate.com/weather/)
.
For each city, extract the following details:

City Name
Temperature
Weather Condition (e.g., Clear, Cloudy, Rainy, etc.)
Store the results in a Pandas DataFrame and export it to a CSV file named weather.csv.

In [25]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

url = "https://www.timeanddate.com/weather/?sort=1"
hdr = {"User-Agent":"Mozilla/5.0"}
res = requests.get(url, headers=hdr)
soup = BeautifulSoup(res.text, "html.parser")

city_names = []
temps = []
conds = []

# main table
tbl = soup.find("table", class_="zebra fw tb-theme")

for r in tbl.find_all("tr")[1:21]:  # top 20 cities
    c = r.find_all("td")
    if len(c) >= 1:
        city = c[0].text.strip()
        link = c[0].find("a")["href"]

        # go to city page
        city_url = "https://www.timeanddate.com" + link
        res2 = requests.get(city_url, headers=hdr)
        soup2 = BeautifulSoup(res2.text, "html.parser")

        # temperature in F
        t = soup2.find("div", class_="h2")
        temp = t.text.strip().replace("\xa0", " ") if t else "NA"

        # weather condition
        w = soup2.find("div", class_="bk-focus__qlook")
        cond = w.find("p").text.strip() if w else "NA"

        city_names.append(city)
        temps.append(temp)
        conds.append(cond)

        time.sleep(1)


df = pd.DataFrame({
    "City": city_names,
    "Temperature (°F)": temps,
    "Condition": conds
})


df.to_csv("weather.csv", index=False)


pd.set_option('display.max_colwidth', None)
print(df)


                                                 City Temperature (°F)  \
0                                    Algeria, Algiers            78 °F   
1                             Argentina, Buenos Aires            45 °F   
2   Australia, Australian Capital Territory, Canberra            45 °F   
3                  Australia, New South Wales, Sydney            61 °F   
4               Australia, Northern Territory, Darwin            81 °F   
5                     Australia, Queensland, Brisbane            67 °F   
6                Australia, South Australia, Adelaide            62 °F   
7                      Australia, Victoria, Melbourne            43 °F   
8                 Australia, Western Australia, Perth            54 °F   
9                           Austria, Vienna, Vienna *            75 °F   
10                                  Bahamas, Nassau *            81 °F   
11                                  Bangladesh, Dhaka            95 °F   
12                                    